# CREDIT CARD APPROVAL PROJECT

This notebook presents a complete machine learning project to predict **credit card approval** using client application data and historical credit behavior.

We'll work with two datasets:

- `application_record.csv`: Contains demographic and financial attributes for each client (one row per client).
- `credit_record.csv`: Contains monthly credit status history per client (multiple rows per client).

The goal is to build a **model** that predicts whether a client should be approved for a credit card, based on their profile and past credit behavior.

### Key steps in this notebook:

1. **Load and inspect the data**
2. **Clean and merge the datasets**
3. **Define a meaningful target variable** (late payment history as a proxy for credit risk for example)
4. **Explore and visualize the data**
5. **Engineer useful features**
6. **Train and evaluate classification models**
7. **Interpret model results and feature importance**

This project will highlight best practices in real-world data science, including dealing with missing values, imbalanced classes, and model evaluation beyond accuracy.


Let's get to it.

In [18]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [19]:
# Now let's load the datasets
df_application = pd.read_csv("data/application_record.csv")
df_credit = pd.read_csv("data/credit_record.csv")

Once we've done this, we're gonna take a look at both dataframes that we've just imported

In [20]:
df_application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,-22717,365243,1,0,0,0,NaN,1.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,-15939,-3007,1,0,0,0,Laborers,1.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,-8169,-372,1,1,0,0,Sales staff,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,-21673,365243,1,0,0,0,NaN,2.0


As we see, it is a big dataset, with 438557 rows and 18 columns, now let's see if it has any missing values...

In [21]:
df_application.isna().sum()

ID                          0
CODE_GENDER                 0
FLAG_OWN_CAR                0
FLAG_OWN_REALTY             0
CNT_CHILDREN                0
AMT_INCOME_TOTAL            0
NAME_INCOME_TYPE            0
NAME_EDUCATION_TYPE         0
NAME_FAMILY_STATUS          0
NAME_HOUSING_TYPE           0
DAYS_BIRTH                  0
DAYS_EMPLOYED               0
FLAG_MOBIL                  0
FLAG_WORK_PHONE             0
FLAG_PHONE                  0
FLAG_EMAIL                  0
OCCUPATION_TYPE        134203
CNT_FAM_MEMBERS             0
dtype: int64

Great, almost every column is free of missing values, just the `OCCUPATION_TYPE` column has. We're going to take care of it later.

Now let's take a look at the other dataset and see how many missing values it has

In [39]:
df_credit.STATUS.value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

This is also a big set, even bigger that the last, with over 1 million rows. Let's look for missing values now...

In [23]:
df_credit.isna().sum()

ID                0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

That is convinient, we have no missing values on this dataset.

Now let's go back and look at the `OCCUPATION_TYPE` column on the first dataset.  
We'll see with what are we dealing with

In [24]:
df_application["OCCUPATION_TYPE"]

0                    NaN
1                    NaN
2         Security staff
3            Sales staff
4            Sales staff
               ...      
438552               NaN
438553          Laborers
438554       Sales staff
438555               NaN
438556       Sales staff
Name: OCCUPATION_TYPE, Length: 438557, dtype: object

In [25]:
df_application["OCCUPATION_TYPE"].value_counts()

OCCUPATION_TYPE
Laborers                 78240
Core staff               43007
Sales staff              41098
Managers                 35487
Drivers                  26090
High skill tech staff    17289
Accountants              15985
Medicine staff           13520
Cooking staff             8076
Security staff            7993
Cleaning staff            5845
Private service staff     3456
Low-skill Laborers        2140
Secretaries               2044
Waiters/barmen staff      1665
Realty agents             1041
HR staff                   774
IT staff                   604
Name: count, dtype: int64

I think the best way to handle this missing values is to fill them with "Unknown" because there's no reason to drop it since it has a lot of values that are not missing

In [26]:
# fill the missing values with "Unknown"
df_application["OCCUPATION_TYPE"] = df_application["OCCUPATION_TYPE"].fillna("Unknown")

In [27]:
# Check if there're still missing values...
df_application.isna().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
dtype: int64

Now that we managed the missing values, let's look at the datatypes of each column in the dataset...

In [28]:
df_application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438557 entries, 0 to 438556
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ID                   438557 non-null  int64  
 1   CODE_GENDER          438557 non-null  object 
 2   FLAG_OWN_CAR         438557 non-null  object 
 3   FLAG_OWN_REALTY      438557 non-null  object 
 4   CNT_CHILDREN         438557 non-null  int64  
 5   AMT_INCOME_TOTAL     438557 non-null  float64
 6   NAME_INCOME_TYPE     438557 non-null  object 
 7   NAME_EDUCATION_TYPE  438557 non-null  object 
 8   NAME_FAMILY_STATUS   438557 non-null  object 
 9   NAME_HOUSING_TYPE    438557 non-null  object 
 10  DAYS_BIRTH           438557 non-null  int64  
 11  DAYS_EMPLOYED        438557 non-null  int64  
 12  FLAG_MOBIL           438557 non-null  int64  
 13  FLAG_WORK_PHONE      438557 non-null  int64  
 14  FLAG_PHONE           438557 non-null  int64  
 15  FLAG_EMAIL       

In [29]:
df_credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   ID              1048575 non-null  int64 
 1   MONTHS_BALANCE  1048575 non-null  int64 
 2   STATUS          1048575 non-null  object
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


The next steps here are preprocess the data to fit it into a model, so lets do that

First, lets look at the first dataset and see what we can do...

In [30]:
df_application.head(20)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Unknown,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
5,5008810,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
6,5008811,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
7,5008812,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,Unknown,1.0
8,5008813,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,Unknown,1.0
9,5008814,F,N,Y,0,283500.0,Pensioner,Higher education,Separated,House / apartment,-22464,365243,1,0,0,0,Unknown,1.0


Here we see that for example, the columns `"DAYS_BIRTH"` and `"DAYS_EMPLOYED"` are negative values, which it's not convenient, we should change it to be positive and improving readability by dividing it by 365. This will be stored in a new column and we'll drop the original ones.  
`"DAYS_BIRTH"`, `"DAYS_EMPLOYED"`: These represent days before the current day
But in `DAYS_EMPLOYED` there's a flag that indicates us that that person is currently unemployed, that's the value **365243**, so we have to handle that before anything.

### Getting data ready

In [31]:
# First we're going to make those values null...
df_application["DAYS_EMPLOYED"] = df_application["DAYS_EMPLOYED"].replace(365243, np.nan) # Turning those values into null
# But we have to keep track of which ones were unemployed before filling it with something, so...
df_application["FLAG_EMPLOYED"] = df_application["DAYS_EMPLOYED"].isna().astype(int)
# Now we can fill those values with the mean of the ones that are valid
df_application["DAYS_EMPLOYED"] = df_application["DAYS_EMPLOYED"].fillna(np.mean(df_application["DAYS_EMPLOYED"]))

In [32]:
# First for "DAYS_BIRTH" which represents AGE
df_application["AGE"] = np.abs(df_application["DAYS_BIRTH"])//365
df_application = df_application.drop("DAYS_BIRTH", axis=1)

# And then for "DAYS_EMPLOYED" which represents SENIORITY
df_application["SENIORITY"] = np.abs(df_application["DAYS_EMPLOYED"])//365
df_application = df_application.drop("DAYS_EMPLOYED", axis=1)

df_application

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,FLAG_EMPLOYED,AGE,SENIORITY
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,1,0,0,Unknown,2.0,0,32,12.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,1,0,0,Unknown,2.0,0,32,12.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1,0,0,0,Security staff,2.0,0,58,3.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,0,52,8.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,0,52,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,M,N,Y,0,135000.0,Pensioner,Secondary / secondary special,Separated,House / apartment,1,0,0,0,Unknown,1.0,1,62,7.0
438553,6840222,F,N,N,0,103500.0,Working,Secondary / secondary special,Single / not married,House / apartment,1,0,0,0,Laborers,1.0,0,43,8.0
438554,6841878,F,N,N,0,54000.0,Commercial associate,Higher education,Single / not married,With parents,1,1,0,0,Sales staff,1.0,0,22,1.0
438555,6842765,F,N,Y,0,72000.0,Pensioner,Secondary / secondary special,Married,House / apartment,1,0,0,0,Unknown,2.0,1,59,7.0


Now we are going to focus on the `CODE_GENDER`, `FLAG_OWN_CAR` and `FLAG_OWN_REALTY` columns and change their values for numeric values such as 0 or 1...

In [33]:
# First we're going to assign the value 0 if it's male and 1 if it's female for 'gender'
df_application["CODE_GENDER"] = df_application["CODE_GENDER"].map({"M": 0, "F": 1})
# Then we're going to assign 1 if it's yes and 0 if it's no for 'own car'
df_application["FLAG_OWN_CAR"] = df_application["FLAG_OWN_CAR"].map({"Y":1, "N":0})
# and the same for 'own realty'
df_application["FLAG_OWN_REALTY"] = df_application["FLAG_OWN_REALTY"].map({"Y":1, "N":0})

In [34]:
# Now we have to turn categorical values into numerical values, so let's use the pandas function 'get_dummies'
# The categorical values are:
# 1. NAME_INCOME_TYPE
# 2. NAME_EDUCATION_TYPE
# 3. NAME_FAMILY_STATUS
# 4. NAME_HOUSING_TYPE
# 5. OCCUPATION_TYPE

df_application = pd.get_dummies(df_application, columns=[
    "NAME_INCOME_TYPE",
    "NAME_EDUCATION_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE",
    "OCCUPATION_TYPE",
], dtype= int)

df_application


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,...,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Unknown,OCCUPATION_TYPE_Waiters/barmen staff
0,5008804,0,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5008805,0,1,1,0,427500.0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5008806,0,1,1,0,112500.0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,5008808,1,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
4,5008809,1,0,1,0,270000.0,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,0,0,1,0,135000.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
438553,6840222,1,0,0,0,103500.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
438554,6841878,1,0,0,0,54000.0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
438555,6842765,1,0,1,0,72000.0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [37]:
df_application.dtypes

ID                                                     int64
CODE_GENDER                                            int64
FLAG_OWN_CAR                                           int64
FLAG_OWN_REALTY                                        int64
CNT_CHILDREN                                           int64
AMT_INCOME_TOTAL                                     float64
FLAG_MOBIL                                             int64
FLAG_WORK_PHONE                                        int64
FLAG_PHONE                                             int64
FLAG_EMAIL                                             int64
CNT_FAM_MEMBERS                                      float64
FLAG_EMPLOYED                                          int64
AGE                                                    int64
SENIORITY                                            float64
NAME_INCOME_TYPE_Commercial associate                  int64
NAME_INCOME_TYPE_Pensioner                             int64
NAME_INCOME_TYPE_State s

Perfect! now all the data is in numeric values, so it's viable to use it to train a machine learning model, but something is missing, and that is our second dataset `df_credit`, and we need that in order to make this model, so let's merge the datasets...

From `df_credit`, we need to create a binary target variable called `bad_credit`, where:

* `1` means the client had any instance of serious delinquency (STATUS in '2', '3', '4', '5').

* `0` means the client never had a serious delinquency.  

Let's do that...

In [44]:
# Let's create this "bad_credit" column and group by its id
df_credit["BAD_CREDIT"] = df_credit["STATUS"].isin(["2","3","4","5"]).astype(int)
credit_agg = df_credit.groupby("ID")["BAD_CREDIT"].max().reset_index()

# Now let's merge the dataframes
df = pd.merge(df_application, credit_agg, on='ID')

df.head(), df.shape

(        ID  CODE_GENDER  FLAG_OWN_CAR  FLAG_OWN_REALTY  CNT_CHILDREN  \
 0  5008804            0             1                1             0   
 1  5008805            0             1                1             0   
 2  5008806            0             1                1             0   
 3  5008808            1             0                1             0   
 4  5008809            1             0                1             0   
 
    AMT_INCOME_TOTAL  FLAG_MOBIL  FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL  ...  \
 0          427500.0           1                1           0           0  ...   
 1          427500.0           1                1           0           0  ...   
 2          112500.0           1                0           0           0  ...   
 3          270000.0           1                0           1           1  ...   
 4          270000.0           1                0           1           1  ...   
 
    OCCUPATION_TYPE_Managers  OCCUPATION_TYPE_Medicine staff  \
 0

In [ ]:
# Check missing values...
df.isna().sum().value_counts()

0    55
Name: count, dtype: int64

The merged dataset has `36,457` rows and `55` columns, so it's time to make and train our model...

## Model Building

Okay we are in a classification problem, but how do we know which model to use?  
Since this question it's imposible to answer without tests, let's test some models ans see how they perform...

In [ ]:
# importing the different models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [48]:
# setting up a random seed
np.random.seed(42)

# Importing and splitting the data into train and test
from sklearn.model_selection import train_test_split

# Split into X & Y
X = df.drop("BAD_CREDIT", axis=1)
y = df["BAD_CREDIT"]

# Split into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [54]:
# Now lets make a dictionary with each model and see which has the best score
# to score these models we're going to use the auc-roc curve and a cross-val-score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

models ={
    "Logistic": LogisticRegression(),
    "RandomForest" : RandomForestClassifier(),
    "KNN": KNeighborsClassifier()
}

for name, model in models.items():
    model.fit(X_train,y_train)
    y_proba = model.predict_proba(X_test)[:,1] # for AUC
    auc = roc_auc_score(y_test, y_proba)
    print(f"{name}: ROC AUC = {auc:.4f}")



Logistic: ROC AUC = 0.5157
RandomForest: ROC AUC = 0.7527
KNN: ROC AUC = 0.7197


Now that we know that the `RandomForestClassifier` it's the best performer, let's work on it

In [55]:
# build the model
clf = RandomForestClassifier()

# fit the data
clf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
# Let's see the model score with the cross_val_score
cross_val_score(clf, X, y, cv = 5)

TypeError: got an unexpected keyword argument 'return_train_score'